# SetUp
Install the dependencies you need to run the notebook

In [9]:
# !brew install poppler tesseract libmagic

## Extract the data

Extract the elements of the PDF that we will be able to use in the retrieval process. These elements can be: Text, Images, Tables, etc.

### Partition PDF tables, text, and images

In [56]:
from unstructured.partition.auto import partition


filename = "inputs/invoices/2Modern.pdf"
elements = partition(filename=filename, content_type="application/pdf")
print("\n\n".join([str(el) for el in elements][:10]))

%6 2MODERN —_

Company Address 35 Miller Ave #192 Invoice 263048 Mill Valley, California 94941 Quote Name PO#24-DPA3-023 United States Created Date 3/25/2025 Submit Payment To: 2Modern PO Box 840603 Los Angeles, CA 90084-0603 Prepared By Pondi Sottile Phone Direct: 925.448.9105 Email pondi@2modern.com Customer PO # PO#24-DPA3-023 Ship To Name Sorensen c/o The Advance Group Bill To Name Hollywood FL Owner, LLC c/o Source Ship To 3715 SW 30th Ave Bill To 921 SW Washington Street, Suite 518 Fort Lauderdale, Florida 33312 Portland, Oregon 97205 United States United States sidemark SM: Source / Diplomat Phase 2B / 24-DPA3-023

/ FB-410 & 24-DPA3-023 / FB-441

Sales Discount Total (Percentage) Price

Product Option/Col tit roduct ption/Color Price Quantity

TAG: FB-410 / Small PI Console Table PI Console Table - Default Title (Small / $999.00 3.00 15.00% | $2,547.45 Mahogany Dark Brown

Castle LED Wall Sconce OTS TAG: FB-441 $2,580.00} 2.00 20.00% | $4,128.00 Smoke / Black / 2 Lights

Specia

In [ ]:
from unstructured.partition.pdf import partition_pdf

input_path = "inputs/invoices/"
file_path = input_path + '2Modern.pdf'
image_output_dir_path = 'outputs/images/'

# Reference: https://docs.unstructured.io/open-source/core-functionality/chunking
chunks = partition_pdf(
    filename=file_path,
    infer_table_structure=True,            # extract tables
    strategy="hi_res",                     # mandatory to infer tables

    extract_image_block_types=["Image", "Table"],   # Add 'Table' to list to extract image of tables
    image_output_dir_path=image_output_dir_path,   # if None, images and tables will saved in base64

    extract_image_block_to_payload=True,   # if true, will extract base64 for API usage

    chunking_strategy="by_title",          # or 'basic'
    max_characters=10000,                  # defaults to 500
    combine_text_under_n_chars=2000,       # defaults to 0
    new_after_n_chars=6000,

    # extract_images_in_pdf=True,          # deprecated
)

ValueError: high_res is not a valid strategy.

In [51]:
chunks[0].metadata.orig_elements

In [53]:
set([str(type(el)) for el in chunks])

{"<class 'unstructured.documents.elements.CompositeElement'>"}

In [44]:
from unstructured.partition.pdf import partition_pdf

fname = "invoices/2Modern.pdf"

elements = partition_pdf(filename=fname,
                         skip_infer_table_types=False,
                         strategy='hi_res',
           )

tables = [el for el in elements if el.category == "Table"]

print(tables[0].text)
print(tables[1].metadata.text_as_html)

Invoice 263048 Quote Name PO#24-DPA3-023 Created Date 3/25/2025
None
